# MNIST 손글씨 숫자 분류 - PyTorch 완전 구현

이 노트북은 PyTorch를 사용하여 MNIST 손글씨 숫자 분류 모델을 처음부터 끝까지 구현하는 실습입니다.

## 목표
- PyTorch를 이용한 완전연결 신경망 구현
- MNIST 데이터셋 로드 및 전처리
- 모델 훈련 및 평가 파이프라인 구축
- 모델 저장/로드 및 실제 이미지 예측

## 학습 내용
1. 데이터 전처리 및 로더 생성
2. 신경망 아키텍처 설계
3. 훈련 루프 구현
4. 모델 평가 및 예측

## 1. 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np

# 한글 폰트 설정 (선택사항)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 2. 신경망 모델 클래스 정의

MNIST 이미지 분류를 위한 완전연결 신경망을 정의합니다.

### 아키텍처:
- 입력층: 784개 뉴런 (28×28 픽셀)
- 은닉층: 500개 뉴런 + ReLU 활성화 함수
- 출력층: 10개 뉴런 (0~9 숫자 클래스)

In [ ]:
class ImageClassifier(nn.Module):
    """MNIST 이미지 분류를 위한 완전연결 신경망"""
    
    def __init__(self, input_size=28*28, hidden_size=500, num_classes=10):
        """
        신경망 레이어 초기화
        Args:
            input_size: 입력 크기 (28x28 = 784)
            hidden_size: 은닉층 크기
            num_classes: 출력 클래스 수 (0~9 숫자)
        """
        super(ImageClassifier, self).__init__()
        
        # 완전연결층 정의
        self.fc1 = nn.Linear(input_size, hidden_size)  # 784 -> 500
        self.relu = nn.ReLU()  # 활성화 함수
        self.fc2 = nn.Linear(hidden_size, num_classes)  # 500 -> 10
        
    def forward(self, x):
        """
        순전파 함수
        Args:
            x: 입력 이미지 텐서 [batch_size, 1, 28, 28]
        Returns:
            출력 텐서 [batch_size, num_classes]
        """
        # 이미지를 1차원 벡터로 평탄화
        x = x.reshape(-1, 28*28)
        
        # 첫 번째 완전연결층과 활성화 함수
        x = self.fc1(x)
        x = self.relu(x)
        
        # 두 번째 완전연결층 (출력층)
        x = self.fc2(x)
        
        return x

print("ImageClassifier 클래스 정의 완료")

## 3. MNIST 분류기 클래스 정의

전체 머신러닝 파이프라인을 관리하는 클래스입니다.

In [ ]:
class MNISTClassifier:
    """MNIST 손글씨 숫자 분류를 위한 클래스"""
    
    def __init__(self, batch_size=64, learning_rate=0.001, epochs=5):
        """
        초기화 함수
        Args:
            batch_size: 배치 크기
            learning_rate: 학습률
            epochs: 에포크 수
        """
        # 하이퍼파라미터 설정
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        # 디바이스 설정 (GPU 사용 가능하면 GPU, 아니면 CPU)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"사용 디바이스: {self.device}")
        
        # 데이터 전처리 변환 정의
        self.transform = transforms.Compose([
            transforms.ToTensor(),  # PIL 이미지를 PyTorch 텐서로 변환 (0~1 사이 값)
            transforms.Normalize((0.5,), (0.5,))  # 평균 0.5, 표준편차 0.5로 정규화
        ])
        
        # 모델, 손실함수, 옵티마이저 초기화
        self.model = None
        self.criterion = None
        self.optimizer = None
        
        # 데이터 로더 초기화
        self.train_loader = None
        self.test_loader = None

print("MNISTClassifier 클래스 정의 완료")

## 4. 데이터 준비 메서드

MNIST 데이터셋을 다운로드하고 데이터 로더를 생성합니다.

In [ ]:
def prepare_data(self):
    """MNIST 데이터셋 준비 및 데이터 로더 생성"""
    print("데이터 준비 중...")
    
    # 훈련 데이터셋 다운로드 및 로드
    train_dataset = datasets.MNIST(
        root="./data", 
        train=True, 
        transform=self.transform, 
        download=True
    )
    self.train_loader = DataLoader(
        dataset=train_dataset, 
        batch_size=self.batch_size, 
        shuffle=True
    )
    
    # 테스트 데이터셋 다운로드 및 로드
    test_dataset = datasets.MNIST(
        root="./data", 
        train=False, 
        transform=self.transform, 
        download=True
    )
    self.test_loader = DataLoader(
        dataset=test_dataset, 
        batch_size=self.batch_size, 
        shuffle=False
    )
    
    print(f"훈련 데이터: {len(train_dataset)}개, 테스트 데이터: {len(test_dataset)}개")

# MNISTClassifier 클래스에 메서드 추가
MNISTClassifier.prepare_data = prepare_data
print("prepare_data 메서드 추가 완료")

## 5. 모델 구축 메서드

신경망 모델, 손실함수, 옵티마이저를 초기화합니다.

In [ ]:
def build_model(self):
    """신경망 모델 생성 및 초기화"""
    print("모델 생성 중...")
    
    # 모델 생성 및 디바이스로 이동
    self.model = ImageClassifier().to(self.device)
    
    # 손실함수 정의 (다중 클래스 분류용)
    self.criterion = nn.CrossEntropyLoss()
    
    # 옵티마이저 정의 (Adam 사용)
    self.optimizer = torch.optim.Adam(
        self.model.parameters(), 
        lr=self.learning_rate
    )
    
    print("모델 생성 완료")
    print(f"모델 파라미터 수: {sum(p.numel() for p in self.model.parameters()):,}")

# MNISTClassifier 클래스에 메서드 추가
MNISTClassifier.build_model = build_model
print("build_model 메서드 추가 완료")

## 6. 훈련 메서드

모델을 훈련시키는 메서드입니다. 배치별 진행상황과 에포크별 평균 손실을 출력합니다.

In [ ]:
def train(self):
    """모델 훈련"""
    print(f"{self.epochs} 에포크 동안 훈련 시작...")
    
    self.model.train()  # 훈련 모드로 설정
    train_losses = []  # 손실 기록용
    
    for epoch in range(self.epochs):
        total_loss = 0
        
        for batch_idx, (images, labels) in enumerate(self.train_loader):
            # 데이터를 디바이스로 이동
            images, labels = images.to(self.device), labels.to(self.device)
            
            # 순전파
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            
            # 역전파
            self.optimizer.zero_grad()  # 기울기 초기화
            loss.backward()  # 역전파 수행
            self.optimizer.step()  # 가중치 업데이트
            
            total_loss += loss.item()
            
            # 진행 상황 출력 (100 배치마다)
            if (batch_idx + 1) % 100 == 0:
                print(f'에포크 [{epoch+1}/{self.epochs}] '
                      f'배치 [{batch_idx+1}/{len(self.train_loader)}] '
                      f'손실: {loss.item():.4f}')
        
        # 에포크별 평균 손실 출력
        avg_loss = total_loss / len(self.train_loader)
        train_losses.append(avg_loss)
        print(f'에포크 {epoch+1} 완료 - 평균 손실: {avg_loss:.4f}')
    
    print("훈련 완료!")
    
    # 손실 그래프 그리기
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, 'b-', linewidth=2)
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Average Loss')
    plt.grid(True)
    plt.show()
    
    return train_losses

# MNISTClassifier 클래스에 메서드 추가
MNISTClassifier.train = train
print("train 메서드 추가 완료")

## 7. 모델 저장/로드 메서드

In [ ]:
def save_model(self, filepath="mnist_model.pth"):
    """훈련된 모델 저장"""
    torch.save(self.model.state_dict(), filepath)
    print(f"모델이 {filepath}에 저장되었습니다.")
    
def load_model(self, filepath="mnist_model.pth"):
    """저장된 모델 불러오기"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"모델 파일 '{filepath}'을 찾을 수 없습니다.")
        
    if self.model is None:
        self.model = ImageClassifier().to(self.device)
        
    self.model.load_state_dict(torch.load(filepath, map_location=self.device))
    self.model.eval()  # 평가 모드로 설정
    print(f"모델이 {filepath}에서 로드되었습니다.")

# MNISTClassifier 클래스에 메서드 추가
MNISTClassifier.save_model = save_model
MNISTClassifier.load_model = load_model
print("save_model, load_model 메서드 추가 완료")

## 8. 모델 평가 메서드

테스트 데이터셋으로 모델의 성능을 평가합니다.

In [ ]:
def evaluate(self):
    """테스트 데이터셋으로 모델 평가"""
    if self.model is None:
        raise ValueError("모델이 초기화되지 않았습니다.")
        
    print("모델 평가 중...")
    self.model.eval()  # 평가 모드로 설정
    
    correct = 0
    total = 0
    class_correct = [0] * 10  # 클래스별 정확도 계산용
    class_total = [0] * 10
    
    with torch.no_grad():  # 기울기 계산 비활성화
        for images, labels in self.test_loader:
            # 데이터를 디바이스로 이동
            images, labels = images.to(self.device), labels.to(self.device)
            
            outputs = self.model(images)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # 클래스별 정확도 계산
            c = (predicted == labels).squeeze()
            for i in range(labels.size(0)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    accuracy = 100 * correct / total
    print(f"전체 테스트 정확도: {accuracy:.2f}% ({correct}/{total})")
    
    # 클래스별 정확도 출력
    print("\n클래스별 정확도:")
    for i in range(10):
        if class_total[i] > 0:
            class_acc = 100 * class_correct[i] / class_total[i]
            print(f"숫자 {i}: {class_acc:.2f}% ({class_correct[i]}/{class_total[i]})")
    
    return accuracy

# MNISTClassifier 클래스에 메서드 추가
MNISTClassifier.evaluate = evaluate
print("evaluate 메서드 추가 완료")

## 9. 단일 이미지 예측 메서드

외부 이미지 파일을 로드하여 예측하는 메서드입니다.

In [ ]:
def predict_image(self, image_path):
    """단일 이미지 예측"""
    if self.model is None:
        raise ValueError("모델이 초기화되지 않았습니다.")
        
    # 이미지 파일 존재 확인
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"이미지 파일 '{image_path}'을 찾을 수 없습니다.")
    
    try:
        # 이미지 로드 및 흑백으로 변환
        image = Image.open(image_path).convert('L')
    except Exception as e:
        raise ValueError(f"이미지 로드 중 오류 발생: {e}")
    
    # 예측용 전처리 (크기 조정 포함)
    predict_transform = transforms.Compose([
        transforms.Resize((28, 28)),  # MNIST 크기로 조정
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # 이미지 전처리 및 배치 차원 추가
    image_tensor = predict_transform(image).unsqueeze(0).to(self.device)
    
    # 예측 수행
    self.model.eval()
    with torch.no_grad():
        output = self.model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    # 결과 시각화
    plt.figure(figsize=(12, 4))
    
    # 원본 이미지
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    # 전처리된 이미지
    plt.subplot(1, 3, 2)
    processed_img = image_tensor.cpu().squeeze().numpy()
    plt.imshow(processed_img, cmap='gray')
    plt.title('Processed Image (28x28)')
    plt.axis('off')
    
    # 예측 확률
    plt.subplot(1, 3, 3)
    probs = probabilities.cpu().numpy()[0]
    plt.bar(range(10), probs)
    plt.title(f'Prediction: {predicted_class} (Confidence: {confidence:.3f})')
    plt.xlabel('Digit')
    plt.ylabel('Probability')
    plt.xticks(range(10))
    
    plt.tight_layout()
    plt.show()
    
    print(f"예측 결과: {predicted_class}")
    print(f"신뢰도: {confidence:.4f}")
    
    return predicted_class, confidence

# MNISTClassifier 클래스에 메서드 추가
MNISTClassifier.predict_image = predict_image
print("predict_image 메서드 추가 완료")

## 10. 데이터 시각화 함수

MNIST 데이터 샘플을 시각화하는 함수입니다.

In [ ]:
def visualize_samples(train_loader, num_samples=8):
    """MNIST 데이터 샘플 시각화"""
    # 첫 번째 배치 가져오기
    data_iter = iter(train_loader)
    images, labels = next(data_iter)
    
    plt.figure(figsize=(12, 6))
    for i in range(num_samples):
        plt.subplot(2, 4, i + 1)
        # 정규화 해제 (0.5 평균, 0.5 표준편차로 정규화되었음)
        img = images[i].squeeze() * 0.5 + 0.5
        plt.imshow(img, cmap='gray')
        plt.title(f'Label: {labels[i].item()}')
        plt.axis('off')
    
    plt.suptitle('MNIST Dataset Samples')
    plt.tight_layout()
    plt.show()

print("visualize_samples 함수 정의 완료")

## 11. 실습 실행

이제 모든 구성요소가 준비되었습니다. 단계별로 실행해보겠습니다.

### 11.1 분류기 인스턴스 생성

In [ ]:
# MNIST 분류기 인스턴스 생성
classifier = MNISTClassifier(batch_size=64, learning_rate=0.001, epochs=3)
print("분류기 인스턴스 생성 완료")

### 11.2 데이터 준비 및 시각화

In [ ]:
# 데이터 준비
classifier.prepare_data()

# 데이터 샘플 시각화
print("\n데이터 샘플 시각화:")
visualize_samples(classifier.train_loader)

### 11.3 모델 생성

In [ ]:
# 모델 생성
classifier.build_model()

# 모델 구조 확인
print("\n모델 구조:")
print(classifier.model)

### 11.4 모델 훈련 (선택적 실행)

**주의**: 이 셀을 실행하면 훈련이 시작됩니다. 시간이 오래 걸릴 수 있습니다.

In [ ]:
# 훈련 실행 (주석 해제하여 사용)
train_losses = classifier.train()
classifier.save_model("mnist_model.pth")

### 11.5 모델 평가 (선택적 실행)

훈련된 모델을 평가합니다. 사전에 훈련된 모델이 있다면 로드하여 사용할 수 있습니다.

In [ ]:
# 모델 평가 (주석 해제하여 사용)
try:
    # 저장된 모델이 있다면 로드
    classifier.load_model("mnist_model.pth")
    accuracy = classifier.evaluate()
except FileNotFoundError:
    print("저장된 모델이 없습니다. 먼저 훈련을 실행하세요.")
except Exception as e:
    print(f"모델 로드 중 오류: {e}")

### 11.6 단일 이미지 예측 (선택적 실행)

외부 이미지 파일로 예측을 수행합니다.

In [ ]:
# 단일 이미지 예측 예시
try:
    # 이미지 경로 설정 (실제 이미지 파일이 있는 경우)
    image_path = './data/mnist_data/1.jpg'  # 예시 경로
    
    # 모델이 로드되지 않았다면 로드
    if classifier.model is None:
        classifier.load_model("mnist_model.pth")
    
    predicted_class, confidence = classifier.predict_image(image_path)
    
except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")
    print("실제 이미지 파일을 준비하거나 경로를 확인하세요.")
except Exception as e:
    print(f"예측 중 오류 발생: {e}")

## 12. 테스트 데이터로 예측 예시

저장된 모델 없이도 테스트할 수 있도록 테스트 데이터에서 몇 개 샘플을 가져와 예측해보겠습니다.

In [ ]:
def test_with_sample_data(classifier, num_samples=5):
    """테스트 데이터에서 샘플을 가져와 예측 수행"""
    if classifier.model is None:
        print("모델이 초기화되지 않았습니다.")
        return
    
    # 테스트 데이터에서 샘플 가져오기
    data_iter = iter(classifier.test_loader)
    images, labels = next(data_iter)
    
    classifier.model.eval()
    
    plt.figure(figsize=(15, 6))
    
    with torch.no_grad():
        # 예측 수행
        images_device = images.to(classifier.device)
        outputs = classifier.model(images_device)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
        
        for i in range(num_samples):
            plt.subplot(2, num_samples, i + 1)
            # 이미지 시각화
            img = images[i].squeeze() * 0.5 + 0.5  # 정규화 해제
            plt.imshow(img, cmap='gray')
            
            actual = labels[i].item()
            pred = predicted[i].item()
            conf = probabilities[i][pred].item()
            
            # 정답과 예측이 같으면 초록색, 다르면 빨간색
            color = 'green' if actual == pred else 'red'
            plt.title(f'Actual: {actual}\nPred: {pred} ({conf:.3f})', color=color)
            plt.axis('off')
            
            # 확률 분포 그래프
            plt.subplot(2, num_samples, i + 1 + num_samples)
            probs = probabilities[i].cpu().numpy()
            bars = plt.bar(range(10), probs)
            bars[pred].set_color('red')  # 예측된 클래스는 빨간색
            if actual != pred:
                bars[actual].set_color('green')  # 실제 클래스는 초록색
            plt.ylim(0, 1)
            plt.xlabel('Digit')
            plt.ylabel('Probability')
            
    plt.tight_layout()
    plt.show()

# 샘플 데이터로 테스트 (모델이 초기화된 경우)
if classifier.model is not None:
    print("테스트 데이터 샘플로 예측 수행:")
    test_with_sample_data(classifier)
else:
    print("모델이 초기화되지 않았습니다. 먼저 build_model()을 실행하거나 훈련된 모델을 로드하세요.")

## 정리 및 핵심 개념

이 실습에서 학습한 내용:

### 🎯 주요 학습 내용

1. **PyTorch 신경망 구조**
   - `nn.Module` 상속을 통한 모델 정의
   - `forward()` 메서드를 통한 순전파 구현
   - 완전연결층(`nn.Linear`)과 활성화 함수(`nn.ReLU`) 사용

2. **데이터 처리 파이프라인**
   - `torchvision.datasets`를 이용한 MNIST 데이터 로드
   - `transforms`를 이용한 데이터 전처리 및 정규화
   - `DataLoader`를 이용한 배치 처리

3. **훈련 루프 구현**
   - 순전파(forward pass) → 손실 계산 → 역전파(backward pass) → 가중치 업데이트
   - `optimizer.zero_grad()`, `loss.backward()`, `optimizer.step()` 패턴

4. **모델 평가 및 예측**
   - `model.eval()`과 `torch.no_grad()` 사용
   - 정확도 계산 및 클래스별 성능 분석
   - 실제 이미지에 대한 예측 수행

### 🔧 주요 PyTorch 개념

- **디바이스 관리**: CPU/GPU 간 텐서 이동
- **모델 모드**: `train()`과 `eval()` 모드 전환
- **손실 함수**: `CrossEntropyLoss`를 이용한 다중 클래스 분류
- **옵티마이저**: Adam을 이용한 가중치 최적화
- **모델 저장/로드**: `state_dict()`를 이용한 모델 영속성

### 📊 성능 개선 방법

- 더 깊은 네트워크 구조 (더 많은 은닉층)
- CNN(Convolutional Neural Network) 사용
- 정규화 기법 (Dropout, Batch Normalization)
- 하이퍼파라미터 튜닝 (학습률, 배치 크기, 에포크 수)
- 데이터 증강 (Data Augmentation)